In [1]:

#!unzip test
!pip install tqdm
#Import everything that is needed from Keras library.
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import glob
#matplotlib will help with displaying the results
import matplotlib.pyplot as plt
#numpy for some mathematical operations
import numpy as np
#PIL for opening,resizing and saving images
from PIL import Image
#tqdm for a progress bar when loading the dataset
from tqdm import tqdm
import scipy as sp
#os library is needed for extracting filenames from the dataset folder.
import os

def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = np.resize(image, new_shape)
        # store
        images_list.append(new_image)
    return np.asarray(images_list)
def get_fid(act1, act2):
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sp.linalg.sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


class FaceGenerator:
    #RGB-images: 3-channels, grayscale: 1-channel, RGBA-images: 4-channels
    def __init__(self,image_width,image_height,channels):
        self.image_width = image_width
        self.image_height = image_height

        self.channels = channels

        self.image_shape = (self.image_width,self.image_height,self.channels)

        #Amount of randomly generated numbers for the first layer of the generator.
        self.random_noise_dimension = 100

        #Just 10 times higher learning rate would result in generator loss being stuck at 0.
        optimizer = Adam(0.0002,0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
        self.generator = self.build_generator()

        #A placeholder for the generator input.
        random_input = Input(shape=(self.random_noise_dimension,))

        #Generator generates images from random noise.
        generated_image = self.generator(random_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        #Discriminator attempts to determine if image is real or generated
        validity = self.discriminator(generated_image)

        #Combined model = generator and discriminator combined.
        #1. Takes random noise as an input.
        #2. Generates an image.
        #3. Attempts to determine if image is real or generated.
        self.combined = Model(random_input,validity)
        self.combined.compile(loss="binary_crossentropy",optimizer=optimizer)

    def get_training_data(self,datafolder):
        print("Loading training data...")

        training_data = []
        #Finds all files in datafolder

        image_list = []
        for filename in glob.glob('*.jpg'): #assuming gif
              
              im=Image.open(filename)
              image = Image.open(filename)
              #Resizes to a desired size.
              image = image.resize((self.image_width,self.image_height),Image.ANTIALIAS)
              #Creates an array of pixel values from the image.
              pixel_array = np.asarray(image)

              training_data.append(pixel_array)
       
        
        
        #training_data is converted to a numpy array
        training_data = np.reshape(training_data,(-1,self.image_width,self.image_height,self.channels))
        return training_data


    def build_generator(self):
        #Generator attempts to fool discriminator by generating new images.
        model = Sequential()

        model.add(Dense(256*4*4,activation="relu",input_dim=self.random_noise_dimension))
        model.add(Reshape((4,4,256)))

        #Four layers of upsampling, convolution, batch normalization and activation.
        # 1. Upsampling: Input data is repeated. Default is (2,2). In that case a 4x4x256 array becomes an 8x8x256 array.
        # 2. Convolution: If you are not familiar, you should watch this video: https://www.youtube.com/watch?v=FTr3n7uBIuE
        # 3. Normalization normalizes outputs from convolution.
        # 4. Relu activation:  f(x) = max(0,x). If x < 0, then f(x) = 0.


        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))


        # Last convolutional layer outputs as many featuremaps as channels in the final image.
        model.add(Conv2D(self.channels,kernel_size=3,padding="same"))
        # tanh maps everything to a range between -1 and 1.
        model.add(Activation("tanh"))

        # show the summary of the model architecture
        model.summary()

        # Placeholder for the random noise input
        input = Input(shape=(self.random_noise_dimension,))
        #Model output
        generated_image = model(input)

        #Change the model type from Sequential to Model (functional API) More at: https://keras.io/models/model/.
        return Model(input,generated_image)


    def build_discriminator(self):
        #Discriminator attempts to classify real and generated images
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.image_shape, padding="same"))
        #Leaky relu is similar to usual relu. If x < 0 then f(x) = x * alpha, otherwise f(x) = x.
        model.add(LeakyReLU(alpha=0.2))

        #Dropout blocks some connections randomly. This help the model to generalize better.
        #0.25 means that every connection has a 25% chance of being blocked.
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        #Zero padding adds additional rows and columns to the image. Those rows and columns are made of zeros.
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        #Flatten layer flattens the output of the previous layer to a single dimension.
        model.add(Flatten())
        #Outputs a value between 0 and 1 that predicts whether image is real or generated. 0 = generated, 1 = real.
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        input_image = Input(shape=self.image_shape)

        #Model output given an image.
        validity = model(input_image)

        return Model(input_image, validity)

    def train(self, datafolder ,epochs,batch_size,save_images_interval):
        #Get the real images
        training_data = self.get_training_data(datafolder)

        #Map all values to a range between -1 and 1.
        training_data = training_data / 127.5 - 1.

        #Two arrays of labels. Labels for real images: [1,1,1 ... 1,1,1], labels for generated images: [0,0,0 ... 0,0,0]
        labels_for_real_images = np.ones((batch_size,1))
        labels_for_generated_images = np.zeros((batch_size,1))
        mfid=[]
        for epoch in range(epochs):
            # Select a random half of images
            
            indices = np.random.randint(0,len(training_data),batch_size)
            real_images = training_data[indices]

            #Generate random noise for a whole batch.
            random_noise = np.random.normal(0,1,(batch_size,self.random_noise_dimension))
            #Generate a batch of new images.
            generated_images = self.generator.predict(random_noise)

            #Train the discriminator on real images.
            discriminator_loss_real = self.discriminator.train_on_batch(real_images,labels_for_real_images)
            #Train the discriminator on generated images.
            discriminator_loss_generated = self.discriminator.train_on_batch(generated_images,labels_for_generated_images)
            #Calculate the average discriminator loss.
            discriminator_loss = 0.5 * np.add(discriminator_loss_real,discriminator_loss_generated)

            #Train the generator using the combined model. Generator tries to trick discriminator into mistaking generated images as real.
            generator_loss = self.combined.train_on_batch(random_noise,labels_for_real_images)
            print ("%d [Discriminator loss: %f, acc.: %.2f%%] [Generator loss: %f]" % (epoch, discriminator_loss[0], 100*discriminator_loss[1], generator_loss))
            images1 = scale_images(real_images, (16,16,3))
            images2 = scale_images(generated_images, (16,16,3))
            act1 = images1.reshape(batch_size,768  )
            act2 = images2.reshape(batch_size,768  )
            temp=(get_fid(act1, act2))
            print('fid : ' + str(temp))
    
            mfid.append(temp)

            if epoch % save_images_interval == 0:
                self.save_images(epoch)

        #Save the model for a later use
        #self.generator.save("saved_models/facegenerator.h5")
        for i in range(0,5000):
            print(mfid[i])


    def save_images(self,epoch):
        #Save 25 generated images for demonstration purposes using matplotlib.pyplot.
        rows, columns = 5, 5
        noise = np.random.normal(0, 1, (rows * columns, self.random_noise_dimension))
        generated_images = self.generator.predict(noise)

        generated_images = 0.5 * generated_images + 0.5

        figure, axis = plt.subplots(rows, columns)
        image_count = 0
        for row in range(rows):
            for column in range(columns):
                axis[row,column].imshow(generated_images[image_count, :], cmap='spring')
                axis[row,column].axis('off')
                image_count += 1
        figure.savefig('./Untitled Folder/generated_' + str(epoch)+ '.png')
        #figure.show()
        plt.close()

    def generate_single_image(self,model_path,image_save_path):
        noise = np.random.normal(0,1,(1,self.random_noise_dimension))
        model = load_model(model_path)
        generated_image = model.predict(noise)
        #Normalized (-1 to 1) pixel values to the real (0 to 256) pixel values.
        generated_image = (generated_image+1)*127.5
        print(generated_image)
        #Drop the batch dimension. From (1,w,h,c) to (w,h,c)
        generated_image = np.reshape(generated_image,self.image_shape)

        image = Image.fromarray(generated_image,"RGB")
        #image.save(image_save_path)

if __name__ == '__main__':
    facegenerator = FaceGenerator(64,64,3)
    facegenerator.train(datafolder="test",epochs=5000, batch_size=32, save_images_interval=100)
    #facegenerator.generate_single_image("saved_models/facegenerator.h5","test.png")
    

You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 17, 17, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 17, 17, 64)        256       
_______________

Loading training data...


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

fid : 48.11226072373181
4803 [Discriminator loss: 0.742622, acc.: 60.94%] [Generator loss: 1.937057]
fid : 44.997428397491944
4804 [Discriminator loss: 0.913049, acc.: 46.88%] [Generator loss: 1.776756]
fid : 57.779151394250746
4805 [Discriminator loss: 0.885128, acc.: 48.44%] [Generator loss: 1.658675]
fid : 43.6483852382154
4806 [Discriminator loss: 0.655246, acc.: 65.62%] [Generator loss: 1.649175]
fid : 31.14906874588233
4807 [Discriminator loss: 0.755286, acc.: 53.12%] [Generator loss: 1.937389]
fid : 76.44717473588207
4808 [Discriminator loss: 0.624828, acc.: 73.44%] [Generator loss: 2.028928]
fid : 47.4478820541926
4809 [Discriminator loss: 0.539857, acc.: 73.44%] [Generator loss: 1.438747]
fid : 44.5483526523155
4810 [Discriminator loss: 0.726727, acc.: 57.81%] [Generator loss: 1.176622]
fid : 34.33657585964294
4811 [Discriminator loss: 0.960891, acc.: 56.25%] [Generator loss: 1.473838]
fid : 36.93840426819638
4812 [Discriminator loss: 0.595472, acc.: 68.75%] [Generator loss: 1

fid : 48.564317499167345
4884 [Discriminator loss: 0.648870, acc.: 64.06%] [Generator loss: 1.460603]
fid : 31.908712286057384
4885 [Discriminator loss: 0.563343, acc.: 68.75%] [Generator loss: 1.664599]
fid : 32.1120291853454
4886 [Discriminator loss: 0.398343, acc.: 85.94%] [Generator loss: 1.888364]
fid : 46.12417176162555
4887 [Discriminator loss: 0.750309, acc.: 59.38%] [Generator loss: 0.970324]
fid : 43.107785674981535
4888 [Discriminator loss: 1.060640, acc.: 29.69%] [Generator loss: 1.459626]
fid : 38.60332644766247
4889 [Discriminator loss: 0.591634, acc.: 78.12%] [Generator loss: 1.470187]
fid : 40.56821757814545
4890 [Discriminator loss: 0.650789, acc.: 68.75%] [Generator loss: 1.355963]
fid : 96.04129497605736
4891 [Discriminator loss: 0.636018, acc.: 62.50%] [Generator loss: 1.381495]
fid : 30.186670544539723
4892 [Discriminator loss: 0.846513, acc.: 56.25%] [Generator loss: 1.483154]
fid : 37.25262306843434
4893 [Discriminator loss: 0.464810, acc.: 75.00%] [Generator los

4965 [Discriminator loss: 0.666582, acc.: 60.94%] [Generator loss: 1.791169]
fid : 40.526206579536996
4966 [Discriminator loss: 0.840562, acc.: 46.88%] [Generator loss: 1.756891]
fid : 68.04783740750935
4967 [Discriminator loss: 1.034951, acc.: 40.62%] [Generator loss: 1.412160]
fid : 45.57468136949748
4968 [Discriminator loss: 1.002941, acc.: 45.31%] [Generator loss: 1.154760]
fid : 41.64965257407502
4969 [Discriminator loss: 0.738970, acc.: 59.38%] [Generator loss: 1.339629]
fid : 45.40784732857129
4970 [Discriminator loss: 0.597095, acc.: 64.06%] [Generator loss: 1.472851]
fid : 44.26950211342168
4971 [Discriminator loss: 0.851611, acc.: 50.00%] [Generator loss: 1.040254]
fid : 58.1966578097789
4972 [Discriminator loss: 0.720305, acc.: 59.38%] [Generator loss: 1.177046]
fid : 45.80677091106773
4973 [Discriminator loss: 0.501230, acc.: 73.44%] [Generator loss: 1.902831]
fid : 61.310532283264976
4974 [Discriminator loss: 0.721515, acc.: 54.69%] [Generator loss: 1.576347]
fid : 47.6738

87.21487718544599
57.0977886376092
86.39351422319376
82.9558510590894
68.9434362740332
42.26691344000125
49.493060541132536
46.306725166781995
55.68767533678661
32.77562277992263
35.266786292221965
75.98431904545578
42.34484320133991
65.19046868947966
51.75780823012816
54.262188975042406
76.78412479744347
102.71042002606913
40.95892904150223
52.38294061457019
87.55659039192624
52.671444725231716
38.20792356686202
47.34386908274952
48.58470220377056
67.83806503132645
72.65364014552271
41.821687865643625
50.51045652797576
46.82842110571918
50.0631936543481
41.084154501976215
61.64779740611425
77.59139728770731
51.68593509438208
68.79185449652736
83.40701961494474
36.40806503205887
81.96623299461714
95.43777810025593
72.96279401920634
86.41048393008967
104.17482163539032
81.34957808059251
88.74893479087257
80.49933226296952
83.7719135539985
69.63251694018768
52.9319487476522
53.42101924532978
50.664195781522906
77.79813290314775
37.67450682583646
65.10430663987454
81.49430439862786
72.796

37.60255453978236
40.82125810947503
49.27491472163926
58.20925739524945
54.82825150761001
42.90919521695845
53.16208316584115
32.502990921613296
64.25399452819693
40.98427184674455
44.846402225841416
42.55338647401617
45.42268940809217
53.6389636169411
38.95114118846668
71.70863334437895
51.85598063256525
40.17250401799261
50.22919811561725
42.675281525676816
36.215776384139936
61.813172287564456
41.85866954505765
52.84363552709455
56.009276425307874
45.82401383740729
66.61215919706139
57.78560879388084
62.282969461354455
50.111618366904665
66.64248169362158
47.78138833914599
71.18027701956991
87.82361722636232
45.08901920621322
52.00675733492116
77.97291045195388
65.65532109112728
64.07854549327521
56.268365915028404
73.02127656036984
49.158335831728245
46.662177173916774
67.14817988473611
59.967539139856186
52.90718932863701
65.15965039037168
63.88774407566335
51.97262332111964
61.27124284640599
56.04579144343375
80.01240425210058
64.88974290643046
59.45844985531761
74.69549681960257